In [1]:
import pandas as pd
import numpy as np
from pyspark.sql import SparkSession


In [2]:
##starting spark session

spark = SparkSession.builder.appName('Logistic').getOrCreate()


22/11/18 22:50:15 WARN Utils: Your hostname, Pardeeps-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.48 instead (on interface en0)
22/11/18 22:50:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/18 22:50:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/18 22:50:16 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
# reading dataset

data = spark.read.csv('cancer.csv', header= True, inferSchema= True)

In [4]:
data.columns

['id',
 'Clump Thickness',
 'UofCSize',
 'UofCShape',
 'Marginal Adhesion',
 'SECSize',
 'Bare Nuclei',
 'Bland Chromatin',
 'Normal Nucleoli',
 'Mitoses',
 'Class']

In [5]:
## assembeling data into feature

from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=['Clump Thickness',
 'UofCSize',
 'UofCShape',
 'Marginal Adhesion',
 'SECSize',
 'Bare Nuclei',
 'Bland Chromatin',
 'Normal Nucleoli',
 'Mitoses',
 'Class'], outputCol='features')


In [6]:
assembler_output = assembler.transform(data)

In [9]:
assembler_output.columns

['id',
 'Clump Thickness',
 'UofCSize',
 'UofCShape',
 'Marginal Adhesion',
 'SECSize',
 'Bare Nuclei',
 'Bland Chromatin',
 'Normal Nucleoli',
 'Mitoses',
 'Class',
 'features']

In [10]:
## spliting data in test and train models

train, test = assembler_output.randomSplit([0.7,0.3])

In [13]:
### setting up logistic model

from pyspark.ml.classification import LogisticRegression
logistic = LogisticRegression(labelCol='Class')


In [15]:
training = logistic.fit(train)

In [17]:
trained = training.evaluate(test)

In [35]:
trained.predictions.columns

['id',
 'Clump Thickness',
 'UofCSize',
 'UofCShape',
 'Marginal Adhesion',
 'SECSize',
 'Bare Nuclei',
 'Bland Chromatin',
 'Normal Nucleoli',
 'Mitoses',
 'Class',
 'features',
 'rawPrediction',
 'probability',
 'prediction']

In [36]:
trained.predictions.select('rawPrediction').show()

+--------------------+
|       rawPrediction|
+--------------------+
|[-4.3314484278528...|
|[-3.7078526803908...|
|[-4.2391503085597...|
|[-4.1111274748864...|
|[-4.1088693377946...|
|[-3.9040139803795...|
|[-4.2113262392292...|
|[-4.1010235155968...|
|[-4.1312121009180...|
|[-4.2301153171970...|
|[-4.2732938731027...|
|[-4.1165917531079...|
|[-4.4389901207373...|
|[-4.1888926048158...|
|[-4.1089140002690...|
|[-4.2280309521182...|
|[-4.1971868857626...|
|[-4.1010235155968...|
|[-4.2603039025183...|
|[-3.6865382762884...|
+--------------------+
only showing top 20 rows



### evaluating model for accuracy

from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator


eval_obj = MulticlassClassificationEvaluator(labelCol='Class')

final_results = eval_obj.evaluate(trained.predictions)

In [43]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
eval = MulticlassClassificationEvaluator(labelCol='Class', predictionCol='prediction', metricName= 'accuracy')
acc = eval.evaluate(trained.predictions)
print(f"Accuracy is {acc}")

Accuracy is 1.0
